In [2]:
from scipy import stats
%matplotlib widget
from scipy.io import loadmat
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sbn
import matplotlib.cm as cm
from Funciones_auxiliares import Mat_to_dataframe,see_teams,get_cluster_indexs,plot_branch,plot_teams,find_mix
from scipy import cluster
from scipy.spatial.distance import mahalanobis

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import scipy as sp
import math

### Importamos la data

In [76]:
plt.close('all')
plt.clf()
plt.close()

data = pd.read_pickle('Datos/PCCIBNMZc')
data['Mean'] = data.Bulk.apply(lambda row: np.mean(row,axis = 0))
data['Mean'] = (data['Mean'] - data.Mean.apply(lambda row:np.mean(row)))/data.Mean.apply(lambda row:np.std(row))
del data['Bulk']
Number_seesions = len(data.PatientExperiment.unique())
print('There are',Number_seesions,' sessions in the data')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

There are 60  sessions in the data


## Separamos en Test y Train por sessiones

## No tengo que entrenar nada, asumimos que el template matching para train es perfecto

Distancia Mahalanobis, necesitamos computar la matriz de covarianza. Dado que asumimos que hay 64 variables la matriz de covarianza.

In [77]:
def Train_test_noise(data):
    Sessions_list = data.PatientExperiment.unique()
    Train_list = np.random.choice(Sessions_list, size=int(len(Sessions_list)/2),replace = False)
    Train = data[data.PatientExperiment.isin(Train_list)]
    Test = data[~data.PatientExperiment.isin(Train_list)]
    Train = Train.reset_index().rename(columns = {'index':'old_index'})
    Test = Test.reset_index().rename(columns = {'index':'old_index'})
    
    df_train = Train.copy()
    df_test = Test.copy()
    
    Noise = df_train[df_train.bNoise == 1]
    Neuron = df_train[df_train.bNoise == 0]
    MU = df_train[df_train.bNoise == 2]

    df_train = pd.concat([Neuron,MU,Noise], ignore_index= True, sort = False)

    Noise = df_test[df_test.bNoise == 1]
    Neuron = df_test[df_test.bNoise == 0]
    MU = df_test[df_test.bNoise == 2]

    df_test = pd.concat([Neuron,MU,Noise], ignore_index= True, sort = False)

    Cov = np.array(df_train.Mean.to_list()).T
    Cov = np.cov(Cov)
    
    return df_train,df_test,Cov

df_train,df_test,Cov = Train_test_noise(data)
print(len(df_train),len(df_test))
print(len(df_train.PatientExperiment.unique()),len(df_test.PatientExperiment.unique()))

753 817
30 30




## Testeamos

In [78]:
from scipy.spatial.distance import squareform, pdist

def compute_dist(df_test,df_train):
    df = pd.concat([df_train,df_test],ignore_index=True,sort = True)
    r = len(df_train)
    Dist = pd.DataFrame(squareform(pdist(np.array(df.loc[:, 'Mean'].to_list()),metric = 'mahalanobis',VI = Cov)),columns=df.index,index=df.index).values
    Dist = Dist[r:,:r]
    return Dist

Dist = compute_dist(df_test,df_train)

Vemos la matriz de distancia, la linea negra separa neuron-noise

In [79]:
fig,ax = plt.subplots()
fig.canvas.draw()

t2 = df_test[df_test.bNoise == 1].index[0]
m2 = df_test[df_test.bNoise == 2].index[0]
t = df_train[df_train.bNoise == 1].index[0]
m = df_train[df_train.bNoise == 2].index[0]

xlabels = ['neuron_train','mu_train','noise_train']
ylabels = ['neuron_test','mu_test','noise_test']

plt.xticks([m/2,(t+m)/2,(len(Dist[0])+t)/2],xlabels)
plt.yticks([m2/2,(t2+m2)/2,(len(Dist)+t2)/2],ylabels)

plt.title('Mahalanobis distance')

plt.hlines(t2,0,len(Dist[0])-1)
plt.hlines(m2,0,len(Dist[0])-1)

plt.vlines(t,0,len(Dist)-1)
plt.vlines(m,0,len(Dist)-1)

plt.imshow(Dist)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

([<matplotlib.axis.XTick at 0x7fd8383bbb90>,
 <a list of 3 Text xticklabel objects>)

([<matplotlib.axis.YTick at 0x7fd823168550>,
 <a list of 3 Text yticklabel objects>)

Text(0.5, 1, 'Mahalanobis distance')

Ahora tenemos una matriz de distancias. Cada fila representa la distancia de un template test a cada uno de los templates de train. Computamos el minimo, y nos fijamos si es neuron o noise, entonces asignamos ese valor.

In [80]:
number_votes = 1
Minn = [np.argsort(i)[:number_votes][0] for i in Dist[:]]
Minn = np.array(Minn)


In [81]:
def create_label(lista,t,m):
    label = np.zeros(len(lista),dtype = int)
    noises_index = np.where(lista >= t)[0]
    mu_index = np.where((lista<t) & (lista>=m))[0]
    for i in noises_index:label[i] = 1
    for i in mu_index:label[i] = 2

    return label
m,t
Minn
label = create_label(Minn,t,m)
label


(152, 250)

array([220, 216,  93,  21,  23,  24,  26, 393, 107,  39,   8,  35, 435,
        37,  38,  39,  40,  41,  42,  43,  40,  38,  45,  46,  47,  47,
        48,  50,  51,  52,  31,  32, 432,  33, 396,  15,  24, 312, 171,
       203,  20, 216, 153,  67,  69,  23, 197,  30,  78,  84,   0,  84,
       128, 246, 204, 116,  74, 240, 137,  77, 178, 148, 116, 104, 199,
       220, 143,  84, 111, 458,  10,  10, 101, 247,  99,  84, 143, 129,
        74, 645, 147,  28,  95,  30, 216, 120, 459,  21, 122, 122,  92,
       132, 144, 229, 448, 148, 106,  87, 618, 105, 108,  63,  99, 349,
       104, 120, 145,  14, 149,  28, 145, 149,   9, 122,  89, 248, 247,
        68, 196, 123, 122,  84,  20, 145,  81,  28, 245, 196, 458,   9,
       115, 126,  14,  84,  69, 104,  23,  86,  25, 557,  25, 144,   8,
       432, 180, 124, 458,  83,  19, 228, 135, 120, 128, 124,  89, 243,
       104,  81,   9,  39,  73, 279,   9, 104,  20, 195, 196,  20, 439,
        22, 200, 180,  37, 737, 643, 102, 243, 240, 164, 644, 34

array([2, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 2, 2, 0, 2, 2, 0,
       0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 2, 0, 0, 2, 0, 0, 0, 2, 2,
       0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 1, 0,
       0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 2, 0, 1, 0, 0, 2, 0, 0, 0, 0,
       0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 2, 0, 1, 0, 2, 2, 0, 1, 1, 0,
       2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 0, 2, 2, 1, 0, 0, 2, 1, 1, 1, 1, 0,
       2, 1, 2, 1, 1, 2, 0, 0, 0, 2, 0, 0, 0, 0, 1, 1, 2, 2, 2, 0, 0, 1,
       2, 0, 1, 2, 2, 0, 0, 0, 0, 0, 1, 0, 1, 1, 2, 2, 1, 2, 0, 0, 1, 1,
       2, 1, 0, 0, 2, 2, 0, 2, 0, 2, 2, 2, 1, 2, 0, 1, 2, 1, 0, 1, 1, 1,
       1, 2, 0, 1, 0, 2, 2, 2, 1, 1, 1, 0, 0, 1, 1, 1, 2, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1,

In [100]:
neurons = label[:m2]
mu = label[m2:t2]
m,t
mu
Minn[m2:t2]
noises = label[t2:]

total_neurons = len(neurons)
total_mu = len(mu)
total_noises = len(noises)

neurons_as_noise = len(np.where(neurons ==1)[0])
neurons_as_mu = len(np.where(neurons ==2)[0])
hits_neuron = len(np.where(neurons ==0)[0])

mu_as_noise = len(np.where(mu == 1)[0])
mu_as_neurons = len(np.where(mu == 0)[0])
hits_mu = len(np.where(mu == 2)[0])

noises_found = sum(noises)

mu_as_noise_index = np.where(mu == 1)[0]
mu_as_noise_index = mu_as_noise_index + m2
noise_matched_index = np.array([Minn[m2+i] for i in mu_as_noise_index]) #notice m2

noise_matched_index
df_train.iloc[noise_matched_index]

print('neurons as noise = ',neurons_as_noise/len(neurons)*100,'({})'.format(neurons_as_noise))
print('neurons as mu = ',neurons_as_mu/len(neurons)*100,'({})'.format(neurons_as_mu))
print('neurons hits = ',hits_neuron/len(neurons)*100,'({})'.format(hits_neuron))

print('\n')

print('mu as noise = ',mu_as_noise/len(mu)*100,'({})'.format(mu_as_noise))
print('mu as neuron = ',mu_as_neurons/len(mu)*100,'({})'.format(mu_as_neurons))
print('mu hits = ',hits_mu/len(mu)*100,'({})'.format(hits_mu))

print('\n')
print('Noise found = ', noises_found/len(noises)*100, '({})'.format(noises_found))

(152, 250)

array([0, 2, 2, 0, 1, 0, 2, 2, 0, 1, 1, 0, 2, 2, 2, 1, 1, 1, 1, 2, 2, 1,
       0, 2, 2, 1, 0, 0, 2, 1, 1, 1, 1, 0, 2, 1, 2, 1, 1, 2, 0, 0, 0, 2,
       0, 0, 0, 0, 1, 1, 2, 2, 2, 0, 0, 1, 2, 0, 1, 2, 2, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 2, 2, 1, 2, 0, 0, 1, 1, 2, 1, 0, 0, 2, 2, 0, 2, 0, 2,
       2, 2, 1, 2, 0, 1, 2, 1, 0, 1, 1, 1, 1, 2, 0, 1, 0, 2])

array([ 20, 195, 196,  20, 439,  22, 200, 180,  37, 737, 643, 102, 243,
       240, 164, 644, 341, 359, 470, 155, 189, 322, 114, 186, 155, 347,
        47,  45, 182, 283, 322, 378, 376,  29, 188, 645, 218, 455, 519,
       212, 129,  83,  77, 229,  74,  92,  56, 145, 645, 555, 222, 207,
       220,  94,  17, 455, 204,  37, 687, 226, 214,  33, 112,  31,  31,
       141, 369,  99, 461, 349, 248, 153, 645, 167,  95,  73, 321, 471,
       240, 369,  77,  32, 234, 230,  24, 202, 137, 153, 167, 221, 519,
       235, 121, 312, 221, 375, 118, 375, 397, 324, 626, 240,  37, 643,
       102, 243])

array([512, 484, 635, 749, 639, 747, 623, 633, 631, 635, 734,  97, 640,
       163, 568, 635, 681, 628, 293, 549, 750, 472, 575, 599, 287, 743,
       746, 744, 473, 730, 632, 516, 653, 632, 434])

,old_index,PatientExperiment,Channel,Cluster,ISI,bNoise,Mean,ZeroCrossings
512,844,HEC006\novel3,92,2,"[0, 114, 180, 543, 313, 192, 188, 130, 99, 102...",1,"[0.07856450930202538, -0.0009784468997115717, ...",1
484,815,HEC006\novel3,82,3,"[0, 2, 4, 10, 34, 31, 29, 20, 11, 11, 13, 11, ...",1,"[0.9252866552215966, 0.7636583741976986, 0.619...",0
635,1282,HEC022\fanis1\001lfpscr_context_pre,70,3,"[0, 2, 6, 16, 3, 5, 4, 0, 1, 3, 3, 6, 3, 1, 6,...",1,"[0.842448915569432, 0.7297510378436408, 0.6777...",5
749,1566,HEC032\HEC032_rsvpscr15,66,2,"[0, 59, 222, 177, 76, 55, 45, 29, 25, 18, 14, ...",1,"[0.9744389761484563, 1.060520982723517, 1.1081...",4
639,1288,HEC022\fanis1\001lfpscr_context_pre,72,3,"[0, 0, 0, 2, 0, 1, 0, 0, 0, 2, 0, 1, 1, 0, 0, ...",1,"[1.5818660140159795, 1.6459865303445451, 1.702...",3
747,1563,HEC032\HEC032_rsvpscr13,81,2,"[0, 309, 15, 76, 1, 26, 0, 12, 3, 3, 2, 3, 3, ...",1,"[0.7993768706034785, 0.6535705808995532, 0.500...",3
623,1266,HEC022\fanis1\001lfpscr_context_pre,65,2,"[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,"[1.6055907307642832, 1.6544604083668422, 1.705...",3
633,1278,HEC022\fanis1\001lfpscr_context_pre,69,3,"[0, 0, 0, 5, 1, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,"[1.5795544434433302, 1.6370186181873525, 1.711...",1
631,1276,HEC022\fanis1\001lfpscr_context_pre,68,3,"[0, 0, 0, 1, 3, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, ...",1,"[1.6566914239313717, 1.7110290567303617, 1.766...",3
635,1282,HEC022\fanis1\001lfpscr_context_pre,70,3,"[0, 2, 6, 16, 3, 5, 4, 0, 1, 3, 3, 6, 3, 1, 6,...",1,"[0.842448915569432, 0.7297510378436408, 0.6777...",5


neurons as noise =  9.75609756097561 (16)
neurons as mu =  14.634146341463413 (24)
neurons hits =  75.60975609756098 (124)


mu as noise =  33.0188679245283 (35)
mu as neuron =  33.0188679245283 (35)
mu hits =  33.9622641509434 (36)


Noise found =  102.01096892138939 (558)


A continuacion los caso para los cuales la distancia entre una neurona y un ruido fueron minimos.. falsos positivos

In [83]:
neurons_as_noise_index = np.where(neurons == 1)[0]
noise_matched_index = [Minn[i] for i in neurons_as_noise_index]

plots_per_fig = 25
number_of_figs = len(neurons_as_noise_index)//plots_per_fig + 1
Figures = [plt.figure() for i in range(number_of_figs)]
axes = [fig.subplots(5,5).flatten() for fig in Figures]
for i,indexes in enumerate(zip(neurons_as_noise_index,noise_matched_index)):
    fig_to_plot = i//plots_per_fig
    ax_to_plot = i%plots_per_fig
    neuron_index,noise_index = indexes
    if noise_index < t:print(noise_index, 'index in df_train is not noise')
    axes[fig_to_plot][ax_to_plot].plot(df_test.iloc[neuron_index].Mean)
    axes[fig_to_plot][ax_to_plot].plot(df_train.iloc[noise_index].Mean)
    axes[fig_to_plot][ax_to_plot].set_title('{}, {}'.format(neuron_index,noise_index))
    
mu_as_noise_index = np.where(mu == 1)[0]
noise_matched_index = [Minn[i] for i in mu_as_noise_index]

plots_per_fig = 25
number_of_figs = len(mu_as_noise_index)//plots_per_fig + 1
Figures = [plt.figure() for i in range(number_of_figs)]
axes = [fig.subplots(5,5).flatten() for fig in Figures]
for i,indexes in enumerate(zip(mu_as_noise_index,noise_matched_index)):
    fig_to_plot = i//plots_per_fig
    ax_to_plot = i%plots_per_fig
    mu_index,noise_index = indexes
    if noise_index < t:print(noise_index, 'index in df_train is not noise')
    axes[fig_to_plot][ax_to_plot].plot(df_test.iloc[mu_index].Mean)
    axes[fig_to_plot][ax_to_plot].plot(df_train.iloc[noise_index].Mean)
    axes[fig_to_plot][ax_to_plot].set_title('{}, {}'.format(mu_index,noise_index))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, '7, 393')

Text(0.5, 1.0, '12, 435')

Text(0.5, 1.0, '32, 432')

Text(0.5, 1.0, '34, 396')

...

105 index in df_train is not noise


Text(0.5, 1.0, '99, 105')

108 index in df_train is not noise


Text(0.5, 1.0, '100, 108')

Text(0.5, 1.0, '103, 349')

In [106]:
def do_one_time(number_votes = 3,report_name = 'report Mahalanobis'):
    df_train,df_test,Cov= Train_test_noise(data)
    
    t2 = df_test[df_test.bNoise == 1].index[0]
    m2 = df_test[df_test.bNoise == 2].index[0]
    t = df_train[df_train.bNoise == 1].index[0]
    m = df_train[df_train.bNoise == 2].index[0]

    neurons_train = m
    mu_train = t-m
    noises_train = len(df_train) - t
    neurons_test = m2
    mu_test = t2-m2
    noises_test = len(df_test) - t2
    
    Dist = compute_dist(df_test,df_train)
    Minn = [np.argsort(i)[:number_votes] for i in Dist[:]]
    label = create_label(Minn,t,m)
    
    neurons = label[:m2]
    mu = label[m2:t2]
    noises = label[t2:]
    
    neurons_as_noise = len(np.where(neurons ==1)[0])
    neurons_as_mu = len(np.where(neurons ==2)[0])
    hits_neuron = len(np.where(neurons ==0)[0])

    mu_as_noise = len(np.where(mu == 1)[0])
    mu_as_neurons = len(np.where(mu == 0)[0])
    hits_mu = len(np.where(mu == 2)[0])

    noises_found = len(np.where(noises == 1)[0]) #Hits noises

    total_found = mu_as_noise + neurons_as_noise + noises_found
    
    neurons_missed = np.where(neurons != 0)[0]
    matched_index_neurons = [Minn[i][0] for i in neurons_missed]
    with open(report_name + '_neu','a') as f:
        for indexes in zip(neurons_missed,matched_index_neurons):
            neurons_index,matched_index = indexes
            old_index_neuron = df_test.iloc[neurons_index].old_index
            old_index_noise = df_train.iloc[matched_index].old_index
            f.write(str(old_index_neuron) + ',' + str(old_index_noise) + '\n')
            
    mu_missed = np.where(mu != 2)[0]
    mu_missed += m2 #notice m2 (is important)
    matched_index_mu = [Minn[m2+i][0] for i in mu_missed]
    with open(report_name + '_mu','a') as f:
        for indexes in zip(mu_missed,matched_index_mu):
            mu_index,matched_index = indexes
            old_index_mu = df_test.iloc[mu_index].old_index
            old_index_noise = df_train.iloc[matched_index].old_index
            f.write(str(old_index_mu) + ',' + str(old_index_noise) + '\n')
            
    
    print('total neurons in test = ',neurons_train,'. This represents ', neurons_train/len(df_test)*100,' of the test size')
    print('total mu in test = ',mu_test,'. This represents ', mu_test/len(df_test)*100,' of the test size')
    print('total noises in test = ',noises_test,'. This represents ', noises_test/len(df_test)*100,' of the test size')

            
    print( 'The algorithm labeled ',total_found, 'clusters as noise')
    print('The algorithm incorrectely labeled',round(neurons_as_noise/neurons_train*100,2), 'of the neurons test as noise')
    print('The algorithm incorrectely labeled',round(mu_as_noise/mu_train*100,2), 'of the mu test as noise')

    print('The algorithm correctely labeled',round(noises_found/noises_test*100,2), 'of the noises test (Hits)')
    
    with open('Mahalanobis_log','a') as f:
                f.write(str(neurons_train) +',' + str(mu_train) +','+str(noises_train) + ',' + str(neurons_test) +',' + str(mu_test) +','+str(noises_test) +',' + str(neurons_as_noise) + ',' + str(neurons_as_mu)+ ',' + str(hits_neuron) +  ',' + str(mu_as_noise)+ ',' + str(mu_as_neurons)+ ',' + str(hits_mu)+','+ str(noises_found) + ','+ str(noises_test - noises_found) + '\n')
        
    print('\n')
    print('\n')

    
    falses_positives = (neurons_as_noise + mu_as_noise)/(neurons_as_noise + mu_as_noise + noises_found)
    hits = noises_found/noises_test
    
    return falses_positives,hits

Realizamos el proceso reiteradas veces para obtener estadistica

In [107]:
falses_positives_list = []
noise_found_list= []
for i in range(20):
    a,b =  do_one_time(number_votes=1)
    falses_positives_list.append(a)
    noise_found_list.append(b)

total neurons in test =  222 . This represents  43.7007874015748  of the test size
total mu in test =  66 . This represents  12.992125984251967  of the test size
total noises in test =  348 . This represents  68.50393700787401  of the test size
The algorithm labeled  315 clusters as noise
The algorithm incorrectely labeled 4.95 of the neurons test as noise
The algorithm incorrectely labeled 11.59 of the mu test as noise
The algorithm correctely labeled 82.76 of the noises test (Hits)




total neurons in test =  162 . This represents  18.32579185520362  of the test size
total mu in test =  134 . This represents  15.158371040723981  of the test size
total noises in test =  596 . This represents  67.42081447963801  of the test size
The algorithm labeled  593 clusters as noise
The algorithm incorrectely labeled 9.26 of the neurons test as noise
The algorithm incorrectely labeled 65.71 of the mu test as noise
The algorithm correctely labeled 89.26 of the noises test (Hits)




total neuron

RESULTADOS

In [108]:
print(np.mean(falses_positives_list),'pm',np.std(falses_positives_list))
print(np.mean(noise_found_list),'pm',np.std(noise_found_list))

0.10738506474578247 pm 0.017667003556190903
0.8457867428986935 pm 0.037764615069256005


## Primero estudiamos los Mu_missed

In [3]:
plt.close('all')
plt.clf()
plt.close()

data = pd.read_pickle('Datos/PCCIBNMZc')
data['Mean'] = data.Bulk.apply(lambda row: np.mean(row,axis = 0))
data['Mean'] = (data['Mean'] - data.Mean.apply(lambda row:np.mean(row)))/data.Mean.apply(lambda row:np.std(row))
Number_seesions = len(data.PatientExperiment.unique())
print('There are',Number_seesions,' sessions in the data')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

There are 60  sessions in the data


In [4]:
def plots_stuff(df,plot_cluster = False,save_name = '',hist_count = []):
    if not hasattr(df, 'Mean_nn'): 
        print('df has not Mean_nn (not normalized) attribute.. computing it from Bulk')
        df['Mean_nn'] = df.Bulk.apply(lambda row:np.mean(row,axis = 0))
    num_figs = len(df)//25+1
    Figures = [plt.figure(i,figsize = (20,10)) for i in range(num_figs)]
    axes = []
    ind_fig = 0
    print('Se realizaran '+str(num_figs) + 'figuras  con '+ str(plots_per_fig) + ' subfiguras cada una... \n')
    print('Figuras finalizadas:')
    for figure in Figures:
        axes.append(figure.subplots(5,5).flat)
        figure.tight_layout()
        figure.subplots_adjust(hspace=.3, wspace=.3)
    for i in range(len(df)):
        Fig_to_plot = i//plots_per_fig
        fig = axes[Fig_to_plot]
        ax_to_plot = i%plots_per_fig
        subplot = fig[ax_to_plot]
        if plot_cluster:
                for j in range(min(len(df.Bulk.iloc[i]),5000)):
                    subplot.plot(df.Bulk.iloc[i][j],'b',linewidth = 0.1)
                subplot.plot(df.Mean_nn.iloc[i],'k',linewidth = 1,label = '{}'.format(i))
        subplot.legend()
        if not hist_count == []:
            subplot.title.set_text(str(hist_count[i]))
        if Fig_to_plot == ind_fig  + 1:
            Figures[ind_fig].savefig(save_name + str(ind_fig))
            plt.close(str(ind_fig))
            ind_fig +=1
            print('\n Figura '+str(ind_fig) +' guardada')
        print(i)
    Figures[-1].savefig(save_name + str(ind_fig + 1))
    print('ultima figura guardada')
    plt.close(str(ind_fig+1))


In [5]:
df = pd.read_csv('report Mahalanobis_mu',names=['mu_missed','matched_index'])
if any(data.iloc[df.mu_missed.values].bNoise == 1): print('There are noises indexes in df.mu_missed')

### Separamos en Mu_missed con neuronas Mu_missed con ruidos. Luego hacemos histogramas

In [6]:
matched_unique_index = df.matched_index.unique()
rows_matched = data.loc[matched_unique_index]
noise_index = rows_matched[rows_matched.bNoise == 1].index
neuron_index = rows_matched[rows_matched.bNoise == 0].index
mu_neuron = df[df.matched_index.isin(neuron_index)]
mu_noise = df[df.matched_index.isin(noise_index)]

In [7]:
hist_mu = np.histogram(df.mu_missed.values,bins = np.arange(len(data)))
plt.figure(figsize = (30,10))
s = np.where(hist_mu[0]>=1)
plt.bar(np.arange(len(s[0])),hist_mu[0][s[0]])
plt.ylabel('counts')
plt.xlabel('index_mu')
plt.title('Mu_missed')

hist_mu_neuron = np.histogram(mu_neuron.mu_missed.values,bins = np.arange(len(data)))
plt.figure(figsize = (30,10))
s = np.where(hist_mu[0]>=1)
plt.bar(np.arange(len(s[0])),hist_mu[0][s[0]])
plt.ylabel('counts')
plt.xlabel('index_mu')
plt.title('Mu_missed (as neuron)')

hist_mu_noise = np.histogram(mu_noise.mu_missed.values,bins = np.arange(len(data)))
plt.figure(figsize = (30,10))
s = np.where(hist_mu[0]>=1)
plt.bar(np.arange(len(s[0])),hist_mu[0][s[0]])
plt.ylabel('counts')
plt.xlabel('index_mu')
plt.title('Mu_missed (as noise)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<Figure size 3000x1000 with 0 Axes>

<BarContainer object of 199 artists>

Text(0, 0.5, 'counts')

Text(0.5, 0, 'index_mu')

...

<Figure size 3000x1000 with 0 Axes>

<BarContainer object of 199 artists>

Text(0, 0.5, 'counts')

Text(0.5, 0, 'index_mu')

Text(0.5, 1.0, 'Mu_missed (as noise)')

Medias de los Mu_missed con neuronas. Luego ploteamos los clusters y guardamos

In [125]:
mu = np.where(hist_mu_neuron[0]>0)[0]
plots_per_fig = 25
number_of_figs = len(mu)//plots_per_fig + 1
Figures = [plt.figure() for i in range(number_of_figs)]
axes = [fig.subplots(5,5).flatten() for fig in Figures]
for i,index_mu in enumerate(mu):
    fig_to_plot = i//plots_per_fig
    ax_to_plot = i%plots_per_fig
    mean_mu = data.iloc[index_mu].Mean
    axes[fig_to_plot][ax_to_plot].plot(mean_mu)
    axes[fig_to_plot][ax_to_plot].set_title('{}'.format(index_mu))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Text(0.5, 1.0, '1350')

Text(0.5, 1.0, '1353')

Text(0.5, 1.0, '1356')

Text(0.5, 1.0, '1360')

Text(0.5, 1.0, '1364')

In [126]:
plt.close('all')
mu_to_plot = data.iloc[mu]
hist_neurons_correlated = hist_mu_neuron[0][mu]
plots_stuff(mu_to_plot,True,'Figuras/mu_missed_mahalanobis/mu_neuron/',hist_count=hist_neurons_correlated)

df has not Mean_nn (not normalized) attribute.. computing it from Bulk


/home/lorenzo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

103


/home/lorenzo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


104


/home/lorenzo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.


ultima figura guardada


Lo mismo para los Mu_missed con ruidos

In [127]:
mu = np.where(hist_mu_noise[0]>0)[0]
plots_per_fig = 25
number_of_figs = len(mu)//plots_per_fig + 1
Figures = [plt.figure() for i in range(number_of_figs)]
axes = [fig.subplots(5,5).flatten() for fig in Figures]
for i,index_mu in enumerate(mu):
    fig_to_plot = i//plots_per_fig
    ax_to_plot = i%plots_per_fig
    mean_mu = data.iloc[index_mu].Mean
    axes[fig_to_plot][ax_to_plot].plot(mean_mu)
    axes[fig_to_plot][ax_to_plot].set_title('{}'.format(index_mu))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Text(0.5, 1.0, '1356')

Text(0.5, 1.0, '1360')

Text(0.5, 1.0, '1361')

Text(0.5, 1.0, '1364')

Text(0.5, 1.0, '1368')

In [128]:
plt.close('all')
mu_to_plot = data.iloc[mu]
hist_neurons_correlated = hist_mu_noise[0][mu]
plots_stuff(mu_to_plot,True,'Figuras/mu_missed_mahalanobis/mu_noise/',hist_count=hist_neurons_correlated)

df has not Mean_nn (not normalized) attribute.. computing it from Bulk


/home/lorenzo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

191


/home/lorenzo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


192


/home/lorenzo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.


ultima figura guardada


### Ahora para el caso de neu_missed (as mu)

In [8]:
df = pd.read_csv('report Mahalanobis_neu',names=['neu_missed','matched_index'])
if any(data.iloc[df.neu_missed.values].bNoise == 1): print('There are noises indexes in df.neu_missed')

In [9]:
matched_unique_index = df.matched_index.unique()
rows_matched = data.loc[matched_unique_index]
noise_index = rows_matched[rows_matched.bNoise == 1].index
mu_index = rows_matched[rows_matched.bNoise == 2].index
neuron_mu = df[df.matched_index.isin(mu_index)]
neuron_noise = df[df.matched_index.isin(noise_index)]

In [10]:
hist_neu = np.histogram(df.neu_missed.values,bins = np.arange(len(data)))
plt.figure(figsize = (30,10))
s = np.where(hist_neu[0]>=1)
plt.bar(np.arange(len(s[0])),hist_neu[0][s[0]])
plt.ylabel('counts')
plt.xlabel('index_mu')
plt.title('Neu_missed')

hist_neu_mu = np.histogram(neuron_mu.neu_missed.values,bins = np.arange(len(data)))
plt.figure(figsize = (30,10))
s = np.where(hist_neu_mu[0]>=1)
plt.bar(np.arange(len(s[0])),hist_neu_mu[0][s[0]])
plt.ylabel('counts')
plt.xlabel('index_mu')
plt.title('neu_missed (as mu)')

hist_neuron_noise = np.histogram(neuron_noise.neu_missed.values,bins = np.arange(len(data)))
plt.figure(figsize = (30,10))
s = np.where(hist_neuron_noise[0]>=1)
plt.bar(np.arange(len(s[0])),hist_neuron_noise[0][s[0]])
plt.ylabel('counts')
plt.xlabel('index_mu')
plt.title('neu_missed (as )')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<Figure size 3000x1000 with 0 Axes>

<BarContainer object of 194 artists>

Text(0, 0.5, 'counts')

Text(0.5, 0, 'index_mu')

...

<Figure size 3000x1000 with 0 Axes>

<BarContainer object of 89 artists>

Text(0, 0.5, 'counts')

Text(0.5, 0, 'index_mu')

Text(0.5, 1.0, 'neu_missed (as )')

In [132]:
neu = np.where(hist_neu_mu[0]>0)[0]
plots_per_fig = 25
number_of_figs = len(neu)//plots_per_fig + 1
Figures = [plt.figure() for i in range(number_of_figs)]
axes = [fig.subplots(5,5).flatten() for fig in Figures]
for i,index_neu in enumerate(neu):
    fig_to_plot = i//plots_per_fig
    ax_to_plot = i%plots_per_fig
    mean_neu = data.iloc[index_neu].Mean
    axes[fig_to_plot][ax_to_plot].plot(mean_neu)
    axes[fig_to_plot][ax_to_plot].set_title('{}'.format(index_neu))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Text(0.5, 1.0, '1485')

Text(0.5, 1.0, '1501')

Text(0.5, 1.0, '1509')

Text(0.5, 1.0, '1514')

Text(0.5, 1.0, '1552')

In [133]:
plt.close('all')
neu_to_plot = data.iloc[neu]
hist_neurons_correlated = hist_neu_mu[0][neu]
plots_stuff(neu_to_plot,True,'Figuras/neu_missed_mahalanobis/neuron_mu/',hist_count=hist_neurons_correlated)

df has not Mean_nn (not normalized) attribute.. computing it from Bulk


/home/lorenzo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

146


/home/lorenzo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


147


/home/lorenzo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.


ultima figura guardada


Lo mismo para los Neu_missed con ruidos

In [76]:
neu = np.where(hist_neuron_noise[0]>0)[0]
plots_per_fig = 25
number_of_figs = len(neu)//plots_per_fig + 1
Figures = [plt.figure() for i in range(number_of_figs)]
axes = [fig.subplots(5,5).flatten() for fig in Figures]
for i,index_neu in enumerate(neu):
    fig_to_plot = i//plots_per_fig
    ax_to_plot = i%plots_per_fig
    mean_neu = data.iloc[index_neu].Mean
    axes[fig_to_plot][ax_to_plot].plot(mean_neu)
    axes[fig_to_plot][ax_to_plot].set_title('{}'.format(index_neu))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Text(0.5, 1.0, '1478')

Text(0.5, 1.0, '1480')

Text(0.5, 1.0, '1483')

Text(0.5, 1.0, '1484')

Text(0.5, 1.0, '1514')

In [63]:
plt.close('all')
neu_to_plot = data.iloc[neu]
hist_neurons_correlated = hist_neu_noise[0][neu]
plots_stuff(neu_to_plot,True,'Figuras/neu_missed_mahalanobis/neuron_noise/',hist_count=hist_neurons_correlated)

df has not Mean_nn (not normalized) attribute.. computing it from Bulk


/home/lorenzo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

234


/home/lorenzo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


235


/home/lorenzo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


236
ultima figura guardada


## Estudiemos un poco mejor los histogramas

Hagamos un nuevo histograma, en el cual estudiamos el numero de neuronas que efectivamente labelearon mu distintas

In [28]:
hist_mu_neuron = {i:0 for i in mu_neuron.matched_index.unique()}
for neuron in hist_mu_neuron: 
    hist_mu_neuron[neuron] = len(mu_neuron[mu_neuron.matched_index == neuron].mu_missed.unique())
    
hist_neuron_mu = {i:0 for i in neuron_mu.matched_index.unique()}
for mu in hist_neuron_mu: 
    hist_neuron_mu[mu] = len(neuron_mu[neuron_mu.matched_index == mu].neu_missed.unique())

neuron_mu

,neu_missed,matched_index
5,486,583
9,529,282
12,536,1070
14,542,1271
20,775,1100
...,...,...
1083,1019,1293
1084,1038,738
1085,1054,738
1089,1478,1356


In [30]:
plt.figure(figsize = (30,10))
plt.bar(np.arange(len(hist_mu_neuron)),hist_mu_neuron.values())

plt.figure(figsize = (30,10))
plt.bar(np.arange(len(hist_neuron_mu)),hist_neuron_mu.values())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<Figure size 3000x1000 with 0 Axes>

<BarContainer object of 142 artists>

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<Figure size 3000x1000 with 0 Axes>

<BarContainer object of 111 artists>

In [29]:
index_neuron = np.where(np.array(list(hist_mu_neuron.values())) > 2)[0]
u = np.array(list(hist_mu_neuron))[index_neuron]
index_mu_to_plot = [mu_to_plot.index.get_loc(i) for i in u]
np.sort(index_mu_to_plot)

NameError: name 'mu_to_plot' is not defined

In [ ]:
hist_mu_index_to_plot = np.array(list(hist_mu_neuron))[index_neuron]
Mu_missed = [mu_neuron[mu_neuron.mu_missed == i].mu_missed.unique() for i in hist_mu_index_to_plot]

In [ ]:
len(Mu_missed)

In [ ]:
plt.close('all')
figs_to_plot = int((len(neurons_near)/2)) + 1
Figures_list = [plt.subplots(2,2,figsize = (10,10)) for i in range(figs_to_plot)]
for i,index_noise in enumerate(index_df_to_plot):
    nfig = i%figs_to_plot
    row_axes = i%2
    subplot_noise = Figures_list[nfig][1][row_axes][0]
    subplot_neuron = Figures_list[nfig][1][row_axes][1]
    subplot_noise.plot(df_to_plot.iloc[index_noise].Mean,label = str(index_noise))
    subplot_noise.legend()
    for index_neuron in neurons_near[i]:
        subplot_neuron.plot(data.loc[index_neuron].Mean,label = str(neurons_to_plot.index.get_loc(index_neuron)))
    subplot_neuron.legend()



Ahora lo mismo pero estudiando las neuronas missed (as_mu)